#### Import statements

In [13]:
import torch
import shutil
import os
import cv2

#### Load the drive folder containing all required files

In [2]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# access the drive folder containing everything we need
%cd /content/drive/My Drive/Colab environments/Risiko! DL

# check that we are in the desired folder
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab environments/Risiko! DL
 3D_models/                                 runs/
 backgrounds/                               Split_train_test_val.ipynb
 coco_risiko.yaml                           synthetic_dataset/
 custom_yolo.yaml                           synthetic_images/
 datasets/                                  tanks_flags_detection.ipynb
 pre_trained_weights/                       Test_detection.ipynb
 real_images/                               test_example.txt
'Risiko!_Synthetic_Dataset_Creator.ipynb'   test.txt
'Risiko! Test.ipynb'                        yolov5/


In [3]:
# generic path to the weights folder
weights_folder = 'runs/train'
weights_path = os.path.join(os.getcwd(), weights_folder)
print(weights_path)

# specific path to weigths obtained with 300 epochs
specific_folder = 'exp_300_epochs/weights/best.pt'
best_weights_path = os.path.join(weights_path, specific_folder)
print(best_weights_path)

/content/drive/My Drive/Colab environments/Risiko! DL/runs/train
/content/drive/My Drive/Colab environments/Risiko! DL/runs/train/exp_300_epochs/weights/best.pt


#### Clone the GitHub repository yolov5 and install requirements

In [4]:
print(os.getcwd())
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

/content/drive/My Drive/Colab environments/Risiko! DL
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/drive/My Drive/Colab environments/Risiko! DL/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.6/612.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


#### Model definition

In [5]:
# set the device
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

# model
print(os.getcwd())
model = torch.hub.load(os.getcwd(), 'custom', path = best_weights_path, source ='local', force_reload=True)
model.to(device)

/content/drive/My Drive/Colab environments/Risiko! DL/yolov5


YOLOv5 🚀 2023-6-11 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

####Intersection over Union (IoU)
We use this method in the section below during the evaluation of the model

In [6]:
# Method to compute the Interection over Union
def calculate_iou(box1, box2):
    # Extract coordinates of the first bounding box
    x1_1, y1_1, x2_1, y2_1, w1_1, w1_2 = box1

    # Extract coordinates of the second bounding box
    x1_2, y1_2, x2_2, y2_2, w2_1, w2_2 = box2

    # Calculate the intersection coordinates
    x_intersection = max(0, min(x2_1, x2_2) - max(x1_1, x1_2))
    y_intersection = max(0, min(y2_1, y2_2) - max(y1_1, y1_2))

    # Calculate the areas of the bounding boxes and the intersection
    area_box1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area_box2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    area_intersection = x_intersection * y_intersection

    # Calculate the union area by adding the individual areas and subtracting the intersection
    area_union = area_box1 + area_box2 - area_intersection

    # Calculate the IoU by dividing the intersection area by the union area
    iou = area_intersection / (area_union + 1e-6)  # Add a small constant to avoid division by zero

    return iou

####Inference on the synthetic test set

In [27]:
# Set the model in evaluation mode
model.eval()

# Path to our dataset
path = '/content/drive/My Drive/Colab environments/Risiko! DL/datasets/test/synthetic/images'

# Get the list of files in the directory
files = os.listdir(path)

# create a list of test images: batch for inference
test_synthetic_images = []

for file_name in files:
    # full file path
    file_path = os.path.join(path, file_name)
    # add the image to the list
    test_synthetic_images.append(file_path)

# inference on the batch of images
results = model(test_synthetic_images, size=640)

####Load the ground truth labels

In [29]:
# path to the labels
labels_path = '/content/drive/My Drive/Colab environments/Risiko! DL/datasets/test/synthetic/labels'

# Get the list of files in the directory
files = os.listdir(labels_path)

# create a list of test images: batch for inference
test_synthetic_labels = []

for file_name in files:
    # full file path
    file_path = os.path.join(labels_path, file_name)
    # add the image to the list
    test_synthetic_labels.append(file_path)

# store the true bboxes values
true_bboxes = []

# store the true class label values
true_labels = []

# iterate over the labels
for label_file in test_synthetic_labels:
    # open the current file
    with open(label_file, "r") as f:
        # class label values in the current file
        current_file_classes = []
        # bboxes values in the current file
        current_file_bboxes = []

        # iterate over the lines: each line is associated with a true instance
        for line in f:
            # extract values from a line
            string_values = line.split()
            # append the class value in the current line of the current file
            current_file_classes.append(float(string_values[0]))
            # values of the bbox in the current line of the current file
            bbox = []
            for i in range(1, len(string_values)):
                bbox.append(float(string_values[i]))
            # append the bbox in the current line of the current file
            current_file_bboxes.append(bbox)

        # append the bboxes related to the current file
        true_bboxes.append(current_file_bboxes)
        # append the class labels related to the current file
        true_bboxes.append(current_file_classes)

####Evaluation

In [33]:
# Perform evaluation on the test set
model.conf = 0.25  # NMS confidence threshold
iou_threshold = 0.45  # IoU threshold for matching bounding boxes
epsilon = 1e-6  # Small constant for numerical stability in F1 score calculation

# iterate over the images
for i in range(len(test_synthetic_images)):

    # result of inference on image $i in the batch
    predictions = results.xyxy[i]

    print(predictions.shape)
    print(len(true_labels[i]))

    # bboxes
    pred_boxes = predictions[:, :4] # [xmin, ymin, xmax, ymax, confidence] for each bbox
    pred_labels = predictions[:, 5] # label for each prediction in the image

    # iterate over the bboxes
    for i in range(len(pred_boxes)):

        # TODO: IL NUMERO DI BBOXES PREDETTE E IL NUMERO DI BBOXES REALI E' DIVERSO,
        # QUINDI E' NECESSARIO CONFRONTARLE UNA AD UNA PER VEDERE QUALI SONO QUELLE CON
        # IOU PIU' ALTO E TENERLE.


        # # Iterate over the predicted boxes
        # for i in range(len(pred_boxes)):
        #     # Extract information for the first predicted box
        #     pred_box_i = pred_boxes[i]
        #     pred_label_i = pred_labels[i]

        #     # Iterate over the remaining predicted boxes
        #     for j in range(i + 1, len(pred_boxes)):
        #         # Extract information for the second predicted box
        #         pred_box_j = pred_boxes[j]
        #         pred_label_j = pred_labels[j]

        #         # Compute the Intersection over Union (IoU) between the two predicted boxes
        #         iou = calculate_iou(pred_box_i, pred_box_j)

        #         # Compute precision, recall, and F1 score based on the IoU
        #         precision = int(pred_label_i == pred_label_j)  # 1 if the predicted labels match, 0 otherwise
        #         recall = int(iou >= iou_threshold)  # 1 if IoU is above the threshold, 0 otherwise
        #         f1_score = 2 * (precision * recall) / (precision + recall + epsilon)  # Compute F1 score (add epsilon to avoid division by zero)

        #         print("Precision:", precision)
        #         print("Recall:", recall)
        #         print("F1 Score:", f1_score)

torch.Size([36, 6])
33
torch.Size([75, 6])
77
torch.Size([54, 6])
53
torch.Size([29, 6])
25
torch.Size([69, 6])
65
torch.Size([90, 6])
79
torch.Size([78, 6])
77
torch.Size([63, 6])
60
torch.Size([33, 6])
33
torch.Size([12, 6])
12
torch.Size([90, 6])
78
torch.Size([82, 6])
80
torch.Size([66, 6])
62
torch.Size([33, 6])
35
torch.Size([76, 6])
75
torch.Size([36, 6])
36
torch.Size([61, 6])
61
torch.Size([86, 6])
85
torch.Size([32, 6])
31
torch.Size([77, 6])
70
torch.Size([19, 6])
18
torch.Size([34, 6])
39
torch.Size([75, 6])
72
torch.Size([75, 6])
72
torch.Size([45, 6])
42
torch.Size([74, 6])
71
torch.Size([53, 6])
51
torch.Size([18, 6])
16
torch.Size([63, 6])
60
torch.Size([52, 6])
50
torch.Size([55, 6])
49
torch.Size([30, 6])
29
torch.Size([88, 6])
75
torch.Size([64, 6])
64
torch.Size([51, 6])
51
torch.Size([23, 6])
22
torch.Size([52, 6])
52
torch.Size([57, 6])
52
torch.Size([83, 6])
74
torch.Size([21, 6])
20
torch.Size([69, 6])
71
torch.Size([68, 6])
64
torch.Size([21, 6])
21
torch.Size(